<a href="https://colab.research.google.com/github/cerr/CERR/blob/octave_dev/Jupyter_Notebooks/demo_DLseg_swallowing_and_chewing_structures.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  Deep learning-based segmentation of Swallowing and Chewing structures in H&N CT scans

This notebook demonstrates the application of Convolution Neural Network based model to segment the following swallowing and chewing structures in H&N CT scans. Model details are provided in [bioRxiv article](https://www.biorxiv.org/content/10.1101/772178v1) by Iyer A *et al*.

### Output structures:

*   Masseters (left, right)
*   Medial pterygoid (left, right)
*   Larynx
*   Constrictor muscle

---


### Install Octave 5.2

In [ ]:
%%capture
! apt-get update
! apt-get install gcc g++ gfortran make libblas-dev liblapack-dev libpcre3-dev libarpack2-dev libcurl4-gnutls-dev epstool libfftw3-dev transfig libfltk1.3-dev libfontconfig1-dev libfreetype6-dev libgl2ps-dev libglpk-dev libreadline-dev gnuplot-x11 libgraphicsmagick++1-dev libhdf5-serial-dev openjdk-8-jdk libsndfile1-dev llvm-dev lpr texinfo libgl1-mesa-dev libosmesa6-dev pstoedit portaudio19-dev libqhull-dev libqrupdate-dev libqscintilla2-dev libsuitesparse-dev texlive texlive-generic-recommended libxft-dev zlib1g-dev autoconf automake bison flex gperf gzip icoutils librsvg2-bin libtool perl rsync tar qtbase5-dev qttools5-dev qttools5-dev-tools libqscintilla2-qt5-dev wget git libsundials-dev gnuplot x11-apps
! pip install octave_kernel
! pip install oct2py

In [ ]:
%%capture
%%bash
LOCATION=$(curl -s https://api.github.com/repos/cerr/octave-colab/releases/latest \
| grep "tag_name" \
| awk '{print "https://github.com/cerr/octave-colab/archive/" substr($2, 2, length($2)-3) ".zip"}') \
; curl -L -o octavecolab.zip $LOCATION

unzip octavecolab.zip -d octavecolab
releaseDir=$(ls octavecolab)
tar xzvf "octavecolab/${releaseDir}/octavecolab.tar.gz"

In [ ]:
import os
os.environ['OCTAVE_EXECUTABLE'] = '/content/octavecolab/bin/octave-cli'
os.environ['PATH'] = '/content/octavecolab/bin:' + os.environ['PATH']

In [ ]:
%%capture
%load_ext oct2py.ipython

!apt install octave liboctave-dev
!wget https://nchc.dl.sourceforge.net/project/octave/Octave%20Forge%20Packages/Individual%20Package%20Releases/image-2.12.0.tar.gz
!wget https://nchc.dl.sourceforge.net/project/octave/Octave%20Forge%20Packages/Individual%20Package%20Releases/io-2.6.1.tar.gz
!wget https://nchc.dl.sourceforge.net/project/octave/Octave%20Forge%20Packages/Individual%20Package%20Releases/statistics-1.4.2.tar.gz

!octave --eval "pkg install image-2.12.0.tar.gz"
!octave --eval "pkg install io-2.6.1.tar.gz"
!octave --eval "pkg install statistics-1.4.2.tar.gz"

### Download the pretrained model

In [ ]:
%%capture
%%bash
LOCATION=$(curl -s https://api.github.com/repos/cerr/CT_SwallowingAndChewing_DeepLabV3/releases/latest \
| grep "tag_name" \
| awk '{print "https://github.com/cerr/CT_SwallowingAndChewing_DeepLabV3/archive/" substr($2, 2, length($2)-3) ".zip"}') \
; curl -L -o CT_swallow_chew.zip $LOCATION

unzip CT_swallow_chew.zip -d CT_SwallowingAndChewing_DeepLabV3
releaseDir=$(ls CT_SwallowingAndChewing_DeepLabV3)

mv CT_SwallowingAndChewing_DeepLabV3/*/* CT_SwallowingAndChewing_DeepLabV3/
rm CT_swallow_chew.zip
rm -r "CT_SwallowingAndChewing_DeepLabV3/$releaseDir"

### Download CERR

In [ ]:
%%capture
%cd /content
!git clone https://github.com/cerr/CERR.git
%cd /content/CERR
!git checkout octave_dev
%cd ..

### Create conda environment for model and its dependencies

In [ ]:
%%capture
!wget https://repo.anaconda.com/miniconda/Miniconda3-py37_4.8.2-Linux-x86_64.sh
!chmod +x Miniconda3-py37_4.8.2-Linux-x86_64.sh
!bash ./Miniconda3-py37_4.8.2-Linux-x86_64.sh -b -f -p /usr/local

In [ ]:
%%capture
!conda env create -f CT_SwallowingAndChewing_DeepLabV3/environment.yml

### Add packages to Octave for i/o and image processing

In [ ]:
%%capture
from oct2py import octave

octave.eval('pkg load image')
octave.eval('pkg load io')
octave.eval('pkg load statistics')
octave.addpath(octave.genpath('/content/CERR'))

### Load a file and apply segmentation model

In [ ]:
%%capture

sampleData = '/content/CERR/Unit_Testing/data_for_cerr_tests/' + \
              'CERR_plans/head_neck_ex1_20may03.mat.bz2'
%octave_push sampleData
planC = octave.loadPlanC(sampleData, octave.tempdir())
planC = octave.updatePlanFields(planC)
planC = octave.quality_assure_planC(sampleData, planC)

algorithm = 'CT_ChewingStructures_DeepLabV3';
sessionPath = '/content/temp';
functionName = '/content/CT_SwallowingAndChewing_DeepLabV3/' + \
               'model_wrapper/CT_ChewingStructures_DeepLabV3/chewing_main.py';
condaEnvName = 'swChewEnv'

planC = octave.runSegForPlanCInCondaEnv(planC,sessionPath,algorithm, \
                                        functionName,[],condaEnvName);

### Display segmentations

In [ ]:
from oct2py import octave
%octave_push planC

In [ ]:
%%octave
indexS = planC{end};
numStructs = length(planC{indexS.structures});
structNameC = {planC{indexS.structures}.structureName}
count = 0;
for strNum = numStructs-3:numStructs
    count = count + 1;
    mask3M = getStrMask(strNum, planC);
    any(mask3M(:))
    maskC{count} = mask3M;
    strNameC{count} = planC{indexS.structures}(strNum).structureName;
end
ctOffset = planC{indexS.scan}(1).scanInfo(1).CTOffset;
scanArray = single(planC{indexS.scan}(1).scanArray) - ctOffset;


structNameC =

{

  [1,1] = Brain

  [1,2] = Brain Stem

  [1,3] = Left Orbit

...

}

ans = 1

ans = 1

ans = 0

ans = 0

In [ ]:
%octave_pull maskC strNameC scanArray

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import clear_output
import ipywidgets as widgets

dx, dy = 1, 1

x = np.arange(0, 255, dx)
y = np.arange(0, 255, dy)
extent = np.min(x), np.max(x), np.min(y), np.max(y)

clear_output(wait=True)

def window_image(image, window_center, window_width):
    img_min = window_center - window_width // 2
    img_max = window_center + window_width // 2
    window_image = image.copy()
    window_image[window_image < img_min] = img_min
    window_image[window_image > img_max] = img_max
    
    return window_image

def show_axial_slice(slcNum):
    clear_output(wait=True)
    print('Slice '+str(slcNum))
    if 'fig' in locals():
        fig.remove()
    fig, ax = plt.subplots()
    window_center = 45
    window_width = 125
    windowed_img = window_image(scanArray[:,:,slcNum-1],
                                window_center,window_width)
    im1 = ax.imshow(windowed_img, cmap=plt.cm.gray, alpha=1,
                    interpolation='nearest', extent=extent)
    cmaps = [plt.cm.Oranges,plt.cm.Oranges,plt.cm.Blues,plt.cm.Blues]
    for maskNum in range(0,4,1):
        mask_cmap = cmaps[maskNum]
        mask_cmap.set_under('k', alpha=0)
        im2 = ax.imshow(maskC[0,maskNum][:,:,slcNum-1], 
                        cmap=mask_cmap, alpha=.8, extent=extent,
                        interpolation='none', clim=[0.5, 1])        
    plt.show()

slice_slider = widgets.IntSlider(min=1,max=79,step=1)
outputSlc = widgets.Output()

display(slice_slider, outputSlc)

def update_slice(change):
    with outputSlc:
        show_axial_slice(change['new'])

slice_slider.observe(update_slice, names='value')


IntSlider(value=1, max=79, min=1)

Output()